# Huspriser data cleaner

In [ ]:
# importer pakker
import numpy as np
import pandas as pd

# Max 10 mio

In [87]:
# importer data
data = pd.read_csv('HousingPrices_original.csv', sep=';')

/var/folders/cv/4hkgm9gn1dv2hp0k2jlt7qrm0000gn/T/ipykernel_4651/1922117324.py:2: DtypeWarning: Columns (31,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('HousingPrices_original.csv', sep=';')


In [88]:
keep_and_rename = {
     'MI_OBJ_OIS_POSTAL_CODE': 'Postnummer',
     'MI_OBJ_OIS_HOUSE_NUMBER': 'Husnummer',
     'MI_OBJ_OIS_SIZE_OF_HOUSE': 'kvadratmeter',
     'MI_OBJ_OIS_SIZE_OF_PLOT': 'Grundareal',
     'MI_OBJ_OIS_CONSTRUCTION_YEAR': 'Byggeår',
     'SCHOOL_DISTANCE_1': 'afstand_skole',
     'SUPERMARKET_DISTANCE_1': 'afstand_supermarked',
     'KOEBESUM_BELOEB': 'Salgspris',
}


In [89]:
existing = {old: new for old, new in keep_and_rename.items() if old in data.columns}
missing  = [old for old in keep_and_rename if old not in data.columns]

subset = data[list(existing.keys())].rename(columns=existing)
subset.isna().sum().sort_values(False)

/var/folders/cv/4hkgm9gn1dv2hp0k2jlt7qrm0000gn/T/ipykernel_4651/2461327743.py:5: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  subset.isna().sum().sort_values(False)


kvadratmeter              0
Salgspris                 0
Husnummer                 4
Grundareal                4
Postnummer               19
afstand_skole            37
afstand_supermarked      37
Byggeår                1024
dtype: int64

In [90]:
data_no_na = subset.dropna()                  
data_no_na.to_csv('HousingPrices_selected.csv', index=False)  
data_no_na.isna().sum().sort_values(False)  

/var/folders/cv/4hkgm9gn1dv2hp0k2jlt7qrm0000gn/T/ipykernel_4651/2844313354.py:3: FutureWarning: In a future version of pandas all arguments of Series.sort_values will be keyword-only.
  data_no_na.isna().sum().sort_values(False)


Postnummer             0
Husnummer              0
kvadratmeter           0
Grundareal             0
Byggeår                0
afstand_skole          0
afstand_supermarked    0
Salgspris              0
dtype: int64

#